In [ ]:
pip install recordlinkage

In [ ]:
import pandas as pd
import recordlinkage

phhs = pd.read_csv('data/Geocoded_Appointment_Data_RiskStratified_2020-04-11_05-05-03.csv')
hhsc = pd.read_csv('data/geocoded_positive_puis_2020-04-10.csv')

In [ ]:
phhs['DOB'] = pd.to_datetime(phhs['DOB'], errors='coerce')
now = pd.to_datetime('now')
phhs['Age'] = (now - phhs['DOB']).astype('<m8[Y]') 

In [ ]:
phhs.head()

In [ ]:
'''#Filtering for only positive results
is_abnormal = phhs['LR_ABNORMALITY'] == 'Abnormal'
print(is_abnormal.head())
phhs_abnormal = phhs[is_abnormal]
print(phhs_abnormal.head())'''

In [ ]:
'''# sorting by patient_mrn 
phhs_abnormal.sort_values("PATIENT_MRN", inplace = True) 
  
# dropping ALL duplicte values 
phhs_abnormal.drop_duplicates(subset ="PATIENT_MRN", 
                     keep = 'first', inplace = True) 
  
# displaying phhs_abnormal
phhs_abnormal'''

In [ ]:
#Setting Index for phhs_abnormal
phhs.set_index("EncounterEpicCsn", inplace = True) 

In [ ]:
#Replacing Values in Column 'Sex'
hhsc_copy = hhsc.replace(to_replace = "F",
                  value = "Female")
hhsc_copy = hhsc_copy.replace(to_replace = "M",
                  value = "Male")
print(hhsc_copy.head())

In [ ]:
'''#Setting Index for hhsc_copy
hhsc_copy.set_index("original_index", inplace = True)'''

In [ ]:
#Creating a full indexer object
'''indexer = recordlinkage.Index()
indexer.full()'''

In [ ]:
#Creating an indexer object with block on state
indexer = recordlinkage.Index()
indexer.block(left_on='Sex', right_on='Sex')
#building up all the potential pairs to check
combinations = indexer.index(phhs, hhsc_copy)
print(len(combinations))

In [ ]:
#Defining the comparison logic
compare = recordlinkage.Compare()
compare.exact('Sex', 'Sex', label = 'Sex')
compare.exact('Age', 'Age (Yrs) at presentation', label = 'Age')
compare.string('geoaddress',
               'geoaddress',
            method='levenshtein',
            threshold=0.95,
            label = 'Address')

In [ ]:
#performing all the feature comparisons using compute
features = compare.compute(combinations, phhs, hhsc_copy)

In [ ]:
#Creating an Indexer object with SortedNeighborhood to add some flexibility for minor spelling mistakes on State
'''indexer = recordlinkage.Index()
indexer.sortedneighbourhood(left_on='STATE', right_on='State')
combinations = indexer.index(phhs_abnormal, hhsc_copy)
print(len(combinations))'''

In [ ]:
#checking for quality of the matches
features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
#adding a total score
potential_matches = features[features.sum(axis=1) >= 0].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'Sex':'Address'].sum(axis=1)

In [ ]:
#phhs_abnormal.loc[5162035,:]

In [ ]:
#hhsc_copy.loc[29,:]

In [ ]:
phhs_abnormal['PHHS_Lookup'] = phhs_abnormal[['Sex Global',
    'ADD_LINE_1', 'CITY', 'STATE'
]].apply(lambda x: '_'.join(x), axis=1)

hhsc_copy['HHSC_Lookup'] = hhsc_copy[['Sex',
    'Address', 'City', 'State' 
]].apply(lambda x: '_'.join(x), axis=1)


In [ ]:
phhs_lookup = phhs_abnormal[['PHHS_Lookup']].reset_index()
hhsc_lookup = hhsc_copy[['HHSC_Lookup']].reset_index()

In [ ]:
phhs_merge = potential_matches.merge(phhs_lookup, how='left')

In [ ]:
final_merge = phhs_merge.merge(hhsc_lookup, how='left')

In [ ]:
'''cols = ['PATIENT_MRN', 'Score',
        'PHHS_Lookup', 'HHSC_Lookup']
final_merge[cols].sort_values(by=['PATIENT_MRN', 'Score'], ascending=False)'''

In [ ]:
#final_merge[final_merge['PATIENT_MRN']==5162035][cols]

In [ ]:
'''final_merge.sort_values(by=['PATIENT_MRN', 'Score'], 
                    ascending=False, inplace = True)
final_merge.drop_duplicates(subset=['PATIENT_MRN'],
                    keep = 'first', inplace = True)'''

In [ ]:
'''s_maxes = final_merge.groupby(['PATIENT_MRN', 'Score']).Score.transform(max)
final_merge = final_merge.loc[final_merge.Score == s_maxes]'''

In [ ]:
final_merge

In [ ]:
final_merge.to_csv('match_merge_list.csv', index=False)